##  immutable 객체 vs mutable 객체
* **immutable** 객체 : **int, string, tuple**
> += 와 + 간의 차이 X    
* **mutable** 객체 : **np.array, list, dictionary**
> += 와 + 간의 차이 ㅇ  
  
***mutable의 = 의 경우, 연산 결과가 새로운 위치에 저장되고, 원본은 그대로 둔 상태에서 새롭게 기존 객체가 가리키는 곳이 업데이트 된다!***

### immutable ex)
+= 와 + 간의 차이 없음

In [1]:
img = 10
print(img, id(img))

img += 20
print(img, id(img))

10 140708915483744
30 140708915484384


In [2]:
img = 10
print(img, id(img))

img = img + 20
print(img, id(img))

10 140708915483744
30 140708915484384


### mutable ex)
+= 와 +의 차이 있음

In [3]:
import numpy as np

img_list = np.array([1, 2, 3])
print(img_list, id(img_list))

img_list += 20
print(img_list, id(img_list)) # += : 같은 공간에 연산 결과 update

[1 2 3] 2557799736864
[21 22 23] 2557799736864


In [4]:
img_list = np.array([1, 2, 3])
print(img_list, id(img_list))

img_list = img_list + 20
print(img_list, id(img_list)) # = : 새로운 공간 생성

[1 2 3] 2557820273968
[21 22 23] 2557820274368


<hr>

# 이벤트 처리
* 키보드 이벤트 처리
* 마우스 이벤트 처리

## 키보드 이벤트 처리

In [5]:
import cv2 as cv

In [6]:
# 특정 키를 눌렀을 때만 창을 닫게끔 / 지정한 시간 동안 키가 눌리지 않았으면 -1 반환
while(True):
    if (cv.waitKey() == 27) : # esc : 27, 다른키 눌렸을 때는 안 꺼지도록 무한 루프
        break

KeyboardInterrupt: 

In [8]:
# (실습) 여러 키 입력에 대해서 서로 다른 처리를 하고 싶은 경우 (i, q)

import cv2

img = cv2.imread('images/lenna.bmp')
if img is None :
    print('Image load failed!')
    exit()

# cf. cv2.namedWindow() : 윈도우를 고정크기로 할지, 사용자가 크기 조절 가능한지 결정
cv2.namedWindow('img', cv2.WINDOW_NORMAL) 
cv2.imshow('img', img)

while True:
    keycode = cv2.waitKey() # 자체적으로 무한 루프 돌면서 키 입력 받아옴
    if keycode == ord('i') or keycode == ord('I'): # i나 I 눌렸을 때
        img = ~img # inverse
        cv2.imshow('img', img)
    elif keycode == 27 or keycode == ord('q') or keycode == ord('Q'): # esc나 q,Q
        break # 종료

cv2.destroyAllWindows()

## 마우스 이벤트 처리
* event : MouseEventType으로 정의된 열거형 상수 중 하나 ==> **이벤트 한번**
* x, y : 마우스 이벤트가 발생한 위치의 좌표
* flags : MouseEventFlags 열거형 상수의 논리합 조합 ==> **뭐가 눌려 있는 상탠지**

## (실습) 마우스 이벤트 처리
* 마우스 왼쪽 버튼을 누르거나 뗸 좌표를 출력
* 왼쪽 버튼을 누른 상태로 마우스를 움직이면 궤적을 따라 영상 위에 노란색으로 표시
* **on_Mouse** 함수를 선언하여 **callback 함수로 지정**

In [9]:
# (실습) 마우스 이벤트 처리
import cv2

# on_mouse : callback 함수
def on_mouse(event, x, y, flags, param):
    global oldx, oldy
    
    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y
        print('EVENT_LBUTTONDOWN: %d, %d' % (x, y))
    
    elif event == cv2.EVENT_LBUTTONUP:
        print('EVENT_LBUTTONUP: %d, %d' % (x, y))
    
    elif event == cv2.EVENT_MOUSEMOVE: # 마우스를 움직이고 있고
        if flags & cv2.EVENT_FLAG_LBUTTON: # 왼쪽 마우스 버튼이 눌린 상태이면
            cv2.line(img, (oldx, oldy), (x, y), (0, 255, 255), 2) # 노란선 그리기
            cv2.imshow('img', img) # 아래 함수 밖에서 선언한 img
            oldx, oldy = x, y
            
            
img = cv2.imread('images/lenna.bmp') 

if img is None:
    print('Image load failed!')
    exit() # 프로그램 전체 종료!
    
cv2.namedWindow('img')
cv2.setMouseCallback('img', on_mouse) # on_mouse()를 마우스 콜백함수로 지정

cv2.imshow('img', img)
cv2.waitKey() # 키보드 입력이 있을 때에만 창 종료
cv2.destroyAllWindows()

EVENT_LBUTTONDOWN: 147, 80
EVENT_LBUTTONUP: 135, 213
EVENT_LBUTTONDOWN: 130, 240
EVENT_LBUTTONUP: 74, 405
EVENT_LBUTTONDOWN: 63, 437
EVENT_LBUTTONUP: 58, 519
EVENT_LBUTTONDOWN: 386, 508
EVENT_LBUTTONUP: 315, 396
EVENT_LBUTTONDOWN: 341, 136
EVENT_LBUTTONUP: 603, 13


## 트랙바 사용하기
* 영상 출력 창의 **상단**에 부착됨
* 필요한 경우 하나에 여러 개의 트랙바 생성 가능
* 각각의 트랙바에 **고유한 이름** 지정해야 함 (트랙바 왼쪽에 이름 나타남)
* **cv2.createTrackbar(트랙바 이름, 트랙바 생성할 창 이름, 시작 위치, 최대 위치, 콜백함수명)**
* cv2.getTrackbarPos() : 트랙바의 현재 위치
* cv2.setTrackbarPos() : 트랙바의 위치를 강제로 특정 위치로 옮기고 싶을 경우

## (실습) 트랙바 사용하기
* 그레이스케일 레벨을 16단계로 보여주기
* 트랙바 위치 (pos)에 16을 곱한 결과를 전체 픽셀 값으로 설정

In [11]:
# (실습) 트랙바 사용하기
import numpy as np
import cv2

def saturated(value):
    # 0보다 작아지거나 255를 넘어버리면 다른 픽셀 값으로 보이므로 예외처리!
    if value > 255:
        value = 255
    elif value < 0: 
        value = 0
        
    return value

# 트랙바 콜백 함수
def on_level_change(pos): # position (현재 트랙바의 위치)
    img[:] = saturated(pos * 16) # numpy 배열의 모든 요소값들은 saturated reuturn값으로
    cv2.imshow('image', img)
    
    
img = np.zeros((400, 400), np.uint8)

cv2.namedWindow('image') # 창 이름 설정
cv2.createTrackbar('level', 'image', 0, 16, on_level_change) # 0 : pos 시작 위치
# 0을 4로 바꾸면 트랙바 위치가 4부터 시작함 (단, 항상 최솟값은 0)

cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

# OpenCV 데이터 파일 입출력
* imwrite() : **uint8(0~255)** 자료형의 데이터만 저장 가능
* 이외에 int, float, double 등의 자료형을 사용하는 일반적인 행렬 ==> **영상 파일로 저장 불가!**
* 이러한 **일반적인 행렬** ==> XML, YAML, JSON 등으로 **저장(write)**하고 **불러오는(read)** 기능 제공 (OpenCV)
    * **FileStorage()** 클래스!

## FileStorage 클래스
* 데이터 파일 입출력 기능들 제공
* **FileStorage 객체**를 생성하여 데이터를 **저장하거나 읽어 옴**
* 생성자의 두 번째 인자 **flags** : **파일 열기 모드** 결정
    * ex) cv2.FILE_STORAGE_READ : 읽기 모드
* isOpened() : 파일이 정상적으로 열렸는지 확인 (T/F)
* release() : 사용 중인 파일 닫고 메모리 버퍼 해제
> 저장이든 읽기든, FileStorage 객체 다 사용했으면 꼭 **release()** 해주기!!!

## (실습) 데이터 파일 저장하기 (write)
* fs = cv2.FileStorage(filename, **cv2.FILE_STORAGE_WRITE**) ==> **쓰기모드로 열기**
* **fs.write()**

In [18]:
import numpy as np
import cv2

filename = 'mydata.json' # json 형식으로 저장하기
# filename = 'mydata.xml'
# filename = 'mydata.yml'

def writeData():
    name = 'Jane'
    age = 10
    pt1 = (100, 200)
    scores = (80, 90, 50)
    mat1 = np.array([[1.0, 1.5], [2.0, 3.2]], dtype = np.float32)
    # uint8만 저장 가능한 imwrite()와 달리, 이번엔 float32 자료형도 저장 가능~!(json)
    
    fs = cv2.FileStorage(filename, cv2.FILE_STORAGE_WRITE) # 쓰기 모드 (생성자)
    
    if not fs.isOpened():
        print('File open failed!')
        return
    
    # (key, value) ==> json 문법
    fs.write('name', name)
    fs.write('age', age)
    fs.write('point', pt1)
    fs.write('scores', scores)
    fs.write('data', mat1)
    
    fs.release() # ***FileStorage 객체 다 사용했으면 꼭 메모리 해제해주기!!***
    
writeData() # mydata.json 생성

## (실습) 데이터 파일 불러오기
* fs = cv2.FileStorage(filename, **cv2.FILE_STORAGE_READ**) ==> **읽기모드로 열기**
* 읽기모드로 열면 파일 전체를 분석하여 계층적 구조를 갖는 노드(node) 집합 구성
    * **노드(node)** : (key, value) 쌍
    * **getNode()**로 특정 이름으로 저장되어 있는 Node에 접근

In [34]:
def readData():    
    fs = cv2.FileStorage(filename, cv2.FILE_STORAGE_READ) # 읽기 모드 (생성자)
    
    if not fs.isOpened():
        print('File open failed!')
        return
    
    name = fs.getNode('name').string() # 그냥 객체(노드)의 string 값 가져오기
    age = int(fs.getNode('age').real())
    pt1 = tuple(fs.getNode('point').mat().astype(np.int32).flatten())
    scores = tuple(fs.getNode('scores').mat().flatten())
    mat1 = fs.getNode('data').mat()
    
    print(fs.getNode('name')) # 뒤에 .string() 안해주면 객체 정보 출력됨
    print(fs.getNode('age').real()) # int로 형변환 안해주면 실수형으로 출력됨(10.0)
    print(fs.getNode('point').mat()) # astype이랑 flatten() 안해주면 2차원 실수 배열
    print()
    
    fs.release() # ***FileStorage 객체 다 사용했으면 꼭 메모리 해제해주기!!***
    
    print('name:', name)
    print('age:', age)
    print('point:', pt1)
    print('scores:', scores)
    print('data:')
    print(mat1)

filename = 'mydata.json'
readData()

<FileNode 0000025388F87E68>
10.0
[[100.]
 [200.]]

name: Jane
age: 10
point: (100, 200)
scores: (80.0, 90.0, 50.0)
data:
[[1.  1.5]
 [2.  3.2]]


# 유용한 OpenCV 기능
* 마스크 연산
* 연산 시간 측정
* NumPy의 sum(), mean(), minMaxLoc(), normalize(), round()

## 마스크 연산
* 임의의 모양을 갖는 ROI(Region-of-Interest) 설정을 위해 마스크 연산
* 보통 입력영상과 크기 같고, **CV_8U (uint8)** 형태로 마스크 지정됨 (0~255)
    * 보통 0 아니면 255 (흑백영상)

## (실습) 마스크 영상
* 영상의 일부 영역에 대해서만 픽셀 값을 노란색으로 설정

In [36]:
def mask_setTo():
    src = cv2.imread('images/lenna.bmp', cv2.IMREAD_COLOR)
    mask = cv2.imread('images/mask_smile.bmp', cv2.IMREAD_GRAYSCALE)
    
    if src is None or mask is None :
        print('Image load failed!')
        return
    
    src[mask > 0] = (0, 255, 255) # (B, G, R) ==> 노란색 (흰부분만 노란색으로)
    
    cv2.imshow('src', src)
    cv2.imshow('mask', mask)
    cv2.waitKey()
    cv2.destroyAllWindows()

mask_setTo()